  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-azokl1jo
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-azokl1jo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.12.0.dev0-py3-none-any.whl size=2943553 sha256=9974030dd52d7227e736f3101fae88d0e2d8816ee66e287e12618a87b48ae7ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-erfqdzdx/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.11.3
    Uninstalling transformers-4.11.3:
      Successfully uninstalled transformers-4.11.3
tokenizers              0.10.3              
transformers            4.12.0.dev0         


In [30]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# パス ['oscar.eo.txt']
paths = ['./tokenizer_training']
print(paths)
# トークナイザーの初期化
tokenizer = ByteLevelBPETokenizer()

# 学習
tokenizer.train(files=paths, vocab_size=5000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

['./tokenizer_training']



CPU times: user 1min 44s, sys: 2.1 s, total: 1min 46s
Wall time: 16.9 s


In [31]:
# モデルの保存
!mkdir EsperBERTo
tokenizer.save_model("EsperBERTo")
tokenizer.save("EsperBERTo/config.json")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: ディレクトリ `EsperBERTo' を作成できません: ファイルが存在します


In [2]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# トークナイザーの準備
tokenizer = ByteLevelBPETokenizer(
    "./EsperBERTo/vocab.json",
    "./EsperBERTo/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
   ("</s>", tokenizer.token_to_id("</s>")),
   ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [3]:
# エンコード
tokenizer.encode("public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound {\n    Argument[] params = new Argument[2];\n    params[0] = new Argument(\"_this\", \"ManagedObjectReference\", _this);\n    params[1] = new Argument(\"users\", \"String[]\", users);\n   ")

Encoding(num_tokens=97, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [4]:
# トークン化
tokenizer.encode("public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound {\n    Argument[] params = new Argument[2];\n    params[0] = new Argument(\"_this\", \"ManagedObjectReference\", _this);\n    params[1] = new Argument(\"users\", \"String[]\", users);\n   ").tokens

['<s>',
 'public',
 'Ġvoid',
 'Ġupdate',
 'Lock',
 'down',
 'Exception',
 's',
 '(',
 'Managed',
 'Object',
 'Reference',
 'Ġ_',
 'this',
 ',',
 'ĠString',
 '[]',
 'Ġuser',
 's',
 ')',
 'Ġthrows',
 'ĠAuth',
 'Min',
 'imum',
 'Admin',
 'Permission',
 ',',
 'ĠRemote',
 'Exception',
 ',',
 'ĠRuntime',
 'F',
 'ault',
 ',',
 'ĠUser',
 'NotFound',
 'Ġ{',
 'Ċ',
 'ĠĠĠ',
 'Ġ',
 'Argument',
 '[]',
 'Ġparams',
 'Ġ=',
 'Ġnew',
 'Ġ',
 'Argument',
 '[',
 '2',
 ']',
 ';',
 'Ċ',
 'ĠĠĠ',
 'Ġparams',
 '[',
 '0',
 ']',
 'Ġ=',
 'Ġnew',
 'Ġ',
 'Argument',
 '("',
 '_',
 'this',
 '",',
 'Ġ"',
 'Managed',
 'Object',
 'Reference',
 '",',
 'Ġ_',
 'this',
 ');',
 'Ċ',
 'ĠĠĠ',
 'Ġparams',
 '[',
 '1',
 ']',
 'Ġ=',
 'Ġnew',
 'Ġ',
 'Argument',
 '("',
 'user',
 's',
 '",',
 'Ġ"',
 'String',
 '[]',
 '",',
 'Ġuser',
 's',
 ');',
 'Ċ',
 'ĠĠĠ',
 '</s>']

In [5]:
# GPUのチェック
!nvidia-smi

Thu Nov  4 18:23:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:09:00.0  On |                  N/A |
|  0%   52C    P8    22W / 220W |    610MiB /  7982MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# PyTorchがGPUを見ていることを確認
import torch
torch.cuda.is_available()

True

In [7]:
from transformers import RobertaConfig

# モデルの設定
config = RobertaConfig(
    model_type="roberta",
    hidden_act="gelu",
    hidden_dropout_prob=0.3,
    attention_probs_dropout_prob=0.1,
    hidden_size=768,
    initalizer_range=0.2,
    vocab_size=5000,
    max_position_embeddings=1024,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
    intermediate_size=4096,
    cache_dir="./cache"
)

In [8]:
from transformers import RobertaTokenizerFast

# トークナイザーの作成
tokenizer = RobertaTokenizerFast.from_pretrained("./EsperBERTo", max_len=512)

In [9]:
from transformers import RobertaForMaskedLM

# モデルの作成
model = RobertaForMaskedLM(config=config)

In [10]:
model.num_parameters()

109166984

In [11]:
%%time
from transformers import LineByLineTextDataset

# データセットの作成
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./tokenizer_training",
    block_size=128,
)

/home/sdlab/.local/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 1min 30s, sys: 1.87 s, total: 1min 32s
Wall time: 13.8 s


In [12]:
from transformers import DataCollatorForLanguageModeling

# data_collatorの作成
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [13]:
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    PreTrainedModel,
    PreTrainedTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)
t_total = len(dataset) // 4 * 50
print(t_total)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=t_total
)

3995950


In [15]:
from transformers import Trainer, TrainingArguments

# トレーナーの引数の作成
training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size= 16,
    save_steps=10_000,
    save_total_limit=2,
    gradient_accumulation_steps = 4
)

# トレーナーの作成
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    optimizers = (optimizer,scheduler)
    #prediction_loss_only=True,
)

In [16]:

trainer.train()

***** Running training *****
  Num examples = 319676
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 74925


Step,Training Loss
500,5.969500
1000,5.430500
1500,5.087000
2000,4.555800
2500,4.082100
3000,3.717800
3500,3.432100
4000,3.194900
4500,2.989000
5000,2.778000


Saving model checkpoint to ./EsperBERTo/checkpoint-10000
Configuration saved in ./EsperBERTo/checkpoint-10000/config.json
Model weights saved in ./EsperBERTo/checkpoint-10000/pytorch_model.bin
Deleting older checkpoint [EsperBERTo/checkpoint-60000] due to args.save_total_limit
Saving model checkpoint to ./EsperBERTo/checkpoint-20000
Configuration saved in ./EsperBERTo/checkpoint-20000/config.json
Model weights saved in ./EsperBERTo/checkpoint-20000/pytorch_model.bin
Deleting older checkpoint [EsperBERTo/checkpoint-70000] due to args.save_total_limit
Saving model checkpoint to ./EsperBERTo/checkpoint-30000
Configuration saved in ./EsperBERTo/checkpoint-30000/config.json
Model weights saved in ./EsperBERTo/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [EsperBERTo/checkpoint-10000] due to args.save_total_limit
Saving model checkpoint to ./EsperBERTo/checkpoint-40000
Configuration saved in ./EsperBERTo/checkpoint-40000/config.json
Model weights saved in ./EsperBERTo/checkpoi

TrainOutput(global_step=74925, training_loss=1.2245778371144582, metrics={'train_runtime': 38647.9211, 'train_samples_per_second': 124.072, 'train_steps_per_second': 1.939, 'total_flos': 3.835502302343274e+17, 'train_loss': 1.2245778371144582, 'epoch': 15.0})

In [17]:
trainer.save_model("./EsperBERTo")

Saving model checkpoint to ./EsperBERTo
Configuration saved in ./EsperBERTo/config.json
Model weights saved in ./EsperBERTo/pytorch_model.bin


In [1]:
from transformers import pipeline

# fill-maskパイプラインの作成
fill_mask = pipeline(
    "fill-mask",
    model="./EsperBERTo",
    tokenizer="./EsperBERTo"
)

In [2]:
# The sun <mask>.
fill_mask("int getValue(){<mask> this.value;}")

[{'sequence': 'int getValue(){ return this.value;}',
  'score': 0.8728135824203491,
  'token': 301,
  'token_str': ' return'},
 {'sequence': 'int getValue(){return this.value;}',
  'score': 0.12403827160596848,
  'token': 3789,
  'token_str': 'return'},
 {'sequence': 'int getValue(){ throw this.value;}',
  'score': 0.0012877093395218253,
  'token': 383,
  'token_str': ' throw'},
 {'sequence': 'int getValue(){= this.value;}',
  'score': 0.0008061031112447381,
  'token': 33,
  'token_str': '='},
 {'sequence': 'int getValue(){  this.value;}',
  'score': 0.000371721776900813,
  'token': 225,
  'token_str': ' '}]

In [3]:
# This is the beginning of a beautiful <mask>.
fill_mask("<mask> getValue(){return this.value;}")

[{'sequence': 'String getValue(){return this.value;}',
  'score': 0.6368172764778137,
  'token': 309,
  'token_str': 'String'},
 {'sequence': 'Object getValue(){return this.value;}',
  'score': 0.14222966134548187,
  'token': 412,
  'token_str': 'Object'},
 {'sequence': 'T getValue(){return this.value;}',
  'score': 0.03950953856110573,
  'token': 56,
  'token_str': 'T'},
 {'sequence': 'E getValue(){return this.value;}',
  'score': 0.03333442658185959,
  'token': 41,
  'token_str': 'E'},
 {'sequence': 'int getValue(){return this.value;}',
  'score': 0.024799060076475143,
  'token': 439,
  'token_str': 'int'}]